In [108]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer
pd.options.display.max_colwidth = 100

In [111]:
def preprocess_news(index,news):
#     print(news)
    wordlist= [word  for sent in nltk.tokenize.sent_tokenize(news) for word in nltk.tokenize.word_tokenize(sent)]
#     print('words in news ', index,' = ', len(wordlist))
    
    #remove stopwords
    stopwordlist = stopwords.words('english')
   
    wordlist = [i for i in wordlist if i not in stopwordlist]
#     print('words after stopword removal in news ', index,' = ', len(wordlist))
    
    # remove words less than or equal to 2 characters
    
    wordlist = [ i for i in wordlist if len(i) >2 ]
#     print('words after small words removal in news ', index,' = ', len(wordlist))
    
    lmtzr = WordNetLemmatizer()
    wordlist = [lmtzr.lemmatize(word) for word in wordlist]
    
    wordlist=set(wordlist)
    preprocessedText= ' '.join(wordlist)
    return preprocessedText

In [112]:
newsdata=pd.read_csv('NlpNewsClassification/bbc-text.csv')

# print(newsdata.shape)
processed_news = []
newscategory = []
for index,row in newsdata.iterrows():
#     print(row['text'])
    newstext = str(row['text'])
    newscategory.append(row['category'])
    
#     print('type(newstext)=',type(newstext))
#     print( newstext)
    processed_news.append(preprocess_news(index,newstext))




# Any results you write to the current directory are saved as output.

In [113]:
# prepare feature set and train test set
featureset=pd.DataFrame()

featureset['text'] = processed_news
featureset['label'] = newscategory
featureset.shape

X_train, X_test, y_train, y_test = train_test_split(featureset['text'],featureset['label'], test_size=0.30, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)



(1557,) (668,) (1557,) (668,)


In [114]:
vectorizer = TfidfVectorizer()
tfIdfVectorizer = vectorizer.fit(featureset['text'])
X_train_tfidf = tfIdfVectorizer.transform(X_train)
X_test_tfidf = tfIdfVectorizer.transform(X_test)

# label encoding
le = preprocessing.LabelEncoder()
le.fit(featureset['label'])
y_train_encoded = le.transform(y_train)
y_test_encoded = le.transform(y_test)

print(X_train_tfidf.shape, X_test_tfidf.shape, y_train_encoded.shape, y_test_encoded.shape)

(1557, 26318) (668, 26318) (1557,) (668,)


In [115]:
def train_model(classifier, feature_vector_train, train_label, feature_vector_valid, valid_label):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, train_label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
        
    return accuracy_score(predictions, valid_label)

In [116]:
#naive bayes
from sklearn.naive_bayes import MultinomialNB
accuracy = train_model(MultinomialNB(), X_train_tfidf, y_train_encoded, X_test_tfidf ,y_test_encoded )
print('NB, Count Vectors: ', accuracy)

NB, Count Vectors:  0.9431137724550899
